In [1]:
try:
# Import necessary libraries
    %pip install pmdarima
    from pmdarima import auto_arima
    # %pip install statsmodels
    from itertools import chain, combinations
    from pyspark.sql import SparkSession
    # from statsmodels.tsa.statespace.sarimax import SARIMAX
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    from datetime import datetime
    from pyspark.dbutils import DBUtils
    import py4j
    import re
    import pyodbc
    import pandas as pd
    import numpy as np
    from pyspark.sql.types import ArrayType, StructType, StructField, StringType
    from pyspark.sql.functions import from_json, col, explode,collect_list
    from sklearn.metrics import mean_squared_error, r2_score
    
    # from pmdarima import auto_arima
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    from datetime import datetime
    from pyspark.dbutils import DBUtils
    import re

    import pyodbc

    import pandas as pd
    import numpy as np

    from pyspark.sql.types import ArrayType, StructType, StructField, StringType 
    from pyspark.sql.functions import from_json, col, explode,collect_list
except Exception as e:
    error_message = str(e)
    #print(f"There is an issue during library instalation : {error_message}")

Note: you may need to restart the kernel to use updated packages.


In [2]:
def wrapper_function(fn):
    def wrapped():
        print("Before function runs")
        result = fn()
        print("After function runs")
        print("----------------")
        return result
    return wrapped

def say_hi():
    message = "Hi!!"
    print(message)
    return message

wrapped_hi = wrapper_function(say_hi)
wrapped_hi()


Before function runs
Hi!!
After function runs
----------------


'Hi!!'

In [3]:
def log_decorator(fn):
    def wrapper():
        print(f"Calling function: {fn.__name__}")
        return fn()
    return wrapper

@log_decorator
def greet():
    message = "Hello!!"
    print(message)
    print("----------------")
    return message

greet()


Calling function: greet
Hello!!
----------------


'Hello!!'

In [4]:
import os

# SQL Server Details
server_name = "jdbc:sqlserver://fortrack-maz-sdb-san-dev-01.database.windows.net"
database_name = "FortrackDB"
url = f"{server_name};databaseName={database_name};"

# SQL Server Tables
table_DBT = "dbo.DataBrickTasks"
table_UFM = "dbo.UserForecastMethod"
table_actual = "dbo.ActualData"
table_version = "dbo.DimVersion"
table_forecast = "dw.ForecastActive"
performance_metrics_table = "dbo.StatisticalPerformanceMetrics"
target_table_name = "dbo.ForecastFact"

# Authentication (Use environment variables for security)
user = os.getenv("DB_USER", "fortrackSQL")
password = os.getenv("DB_PASSWORD", "vuxpapyvu@2024")

# JDBC Driver
jdbc_driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"


In [5]:
import pyodbc
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=fortrack-maz-sdb-san-dev-01.database.windows.net;"
    "DATABASE=FortrackDB;"
    "UID=fortrackSQL;"
    "PWD=vuxpapyvu@2024;"
)
conn = pyodbc.connect(conn_str)

In [6]:
# Create a cursor
cursor = conn.cursor()

# Test the connection
cursor.execute("SELECT TOP 5 * FROM dbo.DataBrickTasks")
rows = cursor.fetchall()
print("ROWS: ", rows)
# Print the results
for row in rows:
    print(row)

# Close the connection
conn.close()


ROWS:  [(1, 'Failed', datetime.datetime(2025, 1, 14, 16, 43, 50, 900000), datetime.datetime(2025, 1, 14, 16, 44, 6, 137000), datetime.datetime(2025, 1, 14, 17, 0, 58, 593000), 137, True, "NameError: name 'selected_columns' is not defined"), (2, 'Completed', datetime.datetime(2025, 1, 14, 16, 44, 53, 683000), datetime.datetime(2025, 1, 14, 17, 1, 4, 457000), datetime.datetime(2025, 1, 14, 19, 50, 19, 957000), 138, True, None), (3, 'Failed', datetime.datetime(2025, 1, 15, 11, 0, 23, 680000), datetime.datetime(2025, 1, 15, 11, 1, 5, 780000), datetime.datetime(2025, 1, 15, 11, 14, 1, 610000), 139, True, "<span class='ansi-red-fg'>Py4JJavaError</span>: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.\n: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 4 times, most recent failure: Lost task 0.3 in stage 10.0 (TID 12) (10.139.64.4 executor 0): com.microsoft.sqlserver.jdbc.SQLServerException: The conversion of the va

In [13]:
try:
    # Import necessary libraries
    from pyspark.sql import SparkSession
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    from itertools import chain, combinations
    import seaborn as sns
    import calendar
    from statsmodels.tsa.arima.model import ARIMA
    from datetime import datetime
    from pyspark.dbutils import DBUtils
    import re
    from sklearn.metrics import mean_squared_error, r2_score
    import pyodbc
    import  py4j

    import pandas as pd
    import numpy as np 
    from pyspark.sql.types import *
    from pyspark.sql.functions import from_json, col, explode,collect_list

except Exception as e:
    error_message = str(e)
    #print(f"There is an issue during library instalation : {error_message}")

In [15]:
databrick_task_id=39
query = f"""
SELECT TOP 1
    ufm.StartDate,
    ufm.EndDate,
    ufm.Parameters,
    ufm.Region,
    ufm.Status,
    ufm.ForecastMethodID,
    ufm.UserForecastMethodID,
    ufm.JSONCustomer as CustomerJSON,
    ufm.varJSON,  
    dfm.Method,
    dbt.DatabrickID
FROM 
    [dbo].[DataBrickTasks] AS dbt
INNER JOIN 
    [dbo].[UserForecastMethod] AS ufm ON dbt.UserForecastMethodID = ufm.UserForecastMethodID
INNER JOIN 
    [dbo].[DimForecastMethod] AS dfm ON ufm.ForecastMethodID = dfm.ForecastMethodID

WHERE  dbt.DatabrickID={databrick_task_id}
ORDER BY
    dbt.CreationDate
"""

In [17]:
import pandas as pd

In [19]:
conn = pyodbc.connect(conn_str)
df = pd.read_sql(query, conn)
df.head()

C:\Users\molef\AppData\Local\Temp\ipykernel_12156\4019385191.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,StartDate,EndDate,Parameters,Region,Status,ForecastMethodID,UserForecastMethodID,CustomerJSON,varJSON,Method,DatabrickID
0,2025-04-01,2026-03-31,"(6,3,1)(6,3,2,3)",EC,Completed,2,64,"{""CustomerID"": [""7999245034"",""7397925811""]}","{""VariableID"": [""OffPeakConsumption"",""PeakCons...",SARIMA,39


In [21]:
Forecast_Method_Name = df.iloc[0]['Method']
Model_Parmeters=df.iloc[0]['Parameters']
UFMID=df.iloc[0]['UserForecastMethodID']
# CustomerID=df.select("Customer").toPandas().iloc[0]['Customer']

StartDate=df.iloc[0]['StartDate']
EndDate=df.iloc[0]['EndDate']
DatabrickID=df.iloc[0]['DatabrickID']
Hyper_Parameters=df.iloc[0]['Parameters']
Forecast_Method_Name, Model_Parmeters,UFMID

('SARIMA', '(6,3,1)(6,3,2,3)', 64)

In [23]:
df['CustomerJSON']

0    {"CustomerID": ["7999245034","7397925811"]}
Name: CustomerJSON, dtype: object

In [25]:
# Check if column exists
if "CustomerJSON" in df.columns:
    customer_ids = []

    # Iterate through each row
    for row in df["CustomerJSON"].dropna():
        try:
            json_list = json.loads(row)
            print("JSON List: ", json_list.items())
            for entry in json_list:
                print("Entry: ",json_list.get(entry))
                customer_ids.extend(json_list.get(entry))
        except json.JSONDecodeError:
            continue  # skip invalid JSON

    # Remove any None values and duplicates
    customer_ids = [cid for cid in customer_ids if cid]
    customer_ids = list(set(customer_ids))

    if customer_ids:
        multiple_customer_ids_str = ",".join([f"'{cid}'" for cid in customer_ids])
    else:
        multiple_customer_ids_str = ''
    
    print("Comma-separated Customer IDs:")
    print(multiple_customer_ids_str)
else:
    print("Column 'CustomerJSON' does not exist in the dataframe.")

JSON List:  dict_items([('CustomerID', ['7999245034', '7397925811'])])
Entry:  ['7999245034', '7397925811']
Comma-separated Customer IDs:
'7397925811','7999245034'


In [27]:
try:
    if "varJSON" in df.columns:
        variable_ids = []

        for row in df["varJSON"].dropna():
            try:
                parsed = json.loads(row)
                print("Parsed: ", parsed)
                vars_list = parsed.get("VariableID", [])
                if vars_list:
                    variable_ids.extend(vars_list)
            except json.JSONDecodeError:
                continue  # skip invalid JSON rows

        # Clean up: remove duplicates and empty entries
        variable_ids = [v for v in variable_ids if v]
        variable_ids = list(set(variable_ids))

        if variable_ids:
            all_variables = ",".join(variable_ids)
        else:
            all_variables = ""

        print("All VariableIDs as comma-separated string:")
        print(all_variables)

    else:
        raise ValueError("Column 'varJSON' does not exist in the dataframe.")
except ValueError as e:
    error_message = str(e)
    raise ValueError("Column 'varJSON' does not exist in the dataframe or schema is invalid.")
except Exception as e:
    error_message = str(e)
    raise RuntimeError(f"Unexpected error occurred: {error_message}")

Parsed:  {'VariableID': ['OffPeakConsumption', 'PeakConsumption', 'StandardConsumption', 'Block1Consumption', 'Block2Consumption', 'Block3Consumption', 'Block4Consumption', 'NonTOUConsumption']}
All VariableIDs as comma-separated string:
Block2Consumption,PeakConsumption,OffPeakConsumption,Block1Consumption,NonTOUConsumption,StandardConsumption,Block3Consumption,Block4Consumption


In [29]:
# Ensure all_variables is a list
if isinstance(all_variables, str):
    all_variables = all_variables.split(',')

In [31]:
all_variables

['Block2Consumption',
 'PeakConsumption',
 'OffPeakConsumption',
 'Block1Consumption',
 'NonTOUConsumption',
 'StandardConsumption',
 'Block3Consumption',
 'Block4Consumption']

In [33]:
all_prediction_columns = ["PeakConsumption", "StandardConsumption", "OffPeakConsumption","Block1Consumption", "Block2Consumption","Block3Consumption", "Block4Consumption",     "NonTOUConsumption"]

all_combination = chain.from_iterable(combinations(all_prediction_columns, r) for r in range(1,len(all_prediction_columns)+1))
columns_mapping = {}
for comb in all_combination:
    comb_set = frozenset(comb)
    comb_value = ['ReportingMonth', 'CustomerID'] + list(comb)
    columns_mapping[comb_set] = comb_value

In [35]:
# Step 4: Print the result for testing
print(f"✅ Total combinations: {len(columns_mapping)}\n")

✅ Total combinations: 255



In [37]:
# Print 5 random combinations
import random
sample_keys = random.sample(list(columns_mapping.keys()), 5)

print("🔍 Sample mappings:")
for k in sample_keys:
    print(f"{set(k)} -> {columns_mapping[k]}")
    print()

🔍 Sample mappings:
{'Block3Consumption', 'StandardConsumption', 'Block1Consumption', 'OffPeakConsumption'} -> ['ReportingMonth', 'CustomerID', 'StandardConsumption', 'OffPeakConsumption', 'Block1Consumption', 'Block3Consumption']

{'StandardConsumption', 'Block1Consumption'} -> ['ReportingMonth', 'CustomerID', 'StandardConsumption', 'Block1Consumption']

{'StandardConsumption', 'Block3Consumption', 'Block4Consumption', 'NonTOUConsumption'} -> ['ReportingMonth', 'CustomerID', 'StandardConsumption', 'Block3Consumption', 'Block4Consumption', 'NonTOUConsumption']

{'StandardConsumption', 'NonTOUConsumption'} -> ['ReportingMonth', 'CustomerID', 'StandardConsumption', 'NonTOUConsumption']

{'Block2Consumption', 'StandardConsumption', 'Block3Consumption', 'Block4Consumption'} -> ['ReportingMonth', 'CustomerID', 'StandardConsumption', 'Block2Consumption', 'Block3Consumption', 'Block4Consumption']



In [39]:
all_variables_set = set(all_variables)
all_variables_set

{'Block1Consumption',
 'Block2Consumption',
 'Block3Consumption',
 'Block4Consumption',
 'NonTOUConsumption',
 'OffPeakConsumption',
 'PeakConsumption',
 'StandardConsumption'}

In [41]:
#print("All selected variables :", str(all_variables_set))

# Find the matching key in the columns_mapping
matching_key = None
# #print ("all_variables_set :", all_variables_set )
for key in columns_mapping.keys():
    # #print(key)
    # #print ("current key from columns mapping :", key )
    if key.issuperset(all_variables_set):
        
        matching_key = key
        break

# Select the appropriate columns based on the matching key
if matching_key:
    # #print(matching_key)
    selected_columns = columns_mapping[matching_key]

    #print(f" All columns based on selectedcolumns{selected_columns}")
else:
    print("No matching columns found in AllVariables")

In [43]:
selected_columns

['ReportingMonth',
 'CustomerID',
 'PeakConsumption',
 'StandardConsumption',
 'OffPeakConsumption',
 'Block1Consumption',
 'Block2Consumption',
 'Block3Consumption',
 'Block4Consumption',
 'NonTOUConsumption']

In [45]:
query = "SELECT * from dbo.PredictiveInputData(64)"
conn = pyodbc.connect(conn_str)
df = pd.read_sql(query, conn)
df.head()

C:\Users\molef\AppData\Local\Temp\ipykernel_12156\2184366618.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,UserForecastMethodID,PodID,CustomerID,TariffID,ReportingMonth,PeakConsumption,StandardConsumption,OffPeakConsumption,Block1Consumption,Block2Consumption,Block3Consumption,Block4Consumption,NonTOUConsumption
0,64,0736171562,7397925811,MEGAFLEX,2021-04-01,141734.0,409040.0,647244.0,0.0,0.0,0.0,0.0,0.0
1,64,0736171562,7397925811,MEGAFLEX,2021-05-01,148250.0,382914.0,561082.0,0.0,0.0,0.0,0.0,0.0
2,64,0736171562,7397925811,MEGAFLEX,2021-06-01,153818.0,394294.0,552320.0,0.0,0.0,0.0,0.0,0.0
3,64,0736171562,7397925811,MEGAFLEX,2021-07-01,182204.0,455822.0,659894.0,0.0,0.0,0.0,0.0,0.0
4,64,0736171562,7397925811,MEGAFLEX,2021-08-01,144534.0,388444.0,621796.0,0.0,0.0,0.0,0.0,0.0


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   UserForecastMethodID  175 non-null    int64  
 1   PodID                 175 non-null    object 
 2   CustomerID            175 non-null    object 
 3   TariffID              175 non-null    object 
 4   ReportingMonth        175 non-null    object 
 5   PeakConsumption       175 non-null    float64
 6   StandardConsumption   175 non-null    float64
 7   OffPeakConsumption    175 non-null    float64
 8   Block1Consumption     175 non-null    float64
 9   Block2Consumption     175 non-null    float64
 10  Block3Consumption     175 non-null    float64
 11  Block4Consumption     175 non-null    float64
 12  NonTOUConsumption     175 non-null    float64
dtypes: float64(8), int64(1), object(4)
memory usage: 17.9+ KB


In [60]:
# --- Optional sorting ---
df = df.sort_values(by=["PodID", "ReportingMonth"])

# --- Type conversion ---
df['CustomerID'] = df['CustomerID'].astype(str)
df['ReportingMonth'] = pd.to_datetime(
    df['ReportingMonth'], format='%Y-%m-%d'
).dt.to_period('M').dt.to_timestamp()

# --- Display top rows ---
display(df.head())

,UserForecastMethodID,PodID,CustomerID,TariffID,ReportingMonth,PeakConsumption,StandardConsumption,OffPeakConsumption,Block1Consumption,Block2Consumption,Block3Consumption,Block4Consumption,NonTOUConsumption
0,64,0736171562,7397925811,MEGAFLEX,2021-04-01,141734.0,409040.0,647244.0,0.0,0.0,0.0,0.0,0.0
1,64,0736171562,7397925811,MEGAFLEX,2021-05-01,148250.0,382914.0,561082.0,0.0,0.0,0.0,0.0,0.0
2,64,0736171562,7397925811,MEGAFLEX,2021-06-01,153818.0,394294.0,552320.0,0.0,0.0,0.0,0.0,0.0
3,64,0736171562,7397925811,MEGAFLEX,2021-07-01,182204.0,455822.0,659894.0,0.0,0.0,0.0,0.0,0.0
4,64,0736171562,7397925811,MEGAFLEX,2021-08-01,144534.0,388444.0,621796.0,0.0,0.0,0.0,0.0,0.0


In [64]:
pandas_df = df.copy()

In [67]:
multiple_customer_ids_list = pandas_df['CustomerID'].unique()
if len(multiple_customer_ids_list)>0:
    # Output the comma-separated IDs
    print(f"Future consumption will be predicted for customers:{multiple_customer_ids_list}")
else:
    # Output the comma-separated IDs
    print(f"No consumption data available for selected Customer IDs: {multiple_customer_ids_list} or customer ids selection is not successful")

# COMMAND ----------

if len(pandas_df) == 0:
    print("No data found for these customers")

Future consumption will be predicted for customers:['7397925811' '7999245034']


In [70]:
# Find the most recent reporting month in the data, which will be used to determine the starting point for forecasting.
Actuals_last_date = pandas_df['ReportingMonth'].max() 
Actuals_last_date

Timestamp('2024-02-01 00:00:00')

In [79]:
StartDate

datetime.date(2025, 4, 1)

In [73]:
# Generate a date range from the start date to the end date with a monthly frequency, starting on the first of each month.
# This range represents the forecast period.

forecast_dates = pd.date_range(start=StartDate, end=EndDate, freq='MS')[0:]
#print(forecast_dates,f"Actual_last_date {Actuals_last_date}",f"No of month to pr

In [77]:
list(forecast_dates)

[Timestamp('2025-04-01 00:00:00'),
 Timestamp('2025-05-01 00:00:00'),
 Timestamp('2025-06-01 00:00:00'),
 Timestamp('2025-07-01 00:00:00'),
 Timestamp('2025-08-01 00:00:00'),
 Timestamp('2025-09-01 00:00:00'),
 Timestamp('2025-10-01 00:00:00'),
 Timestamp('2025-11-01 00:00:00'),
 Timestamp('2025-12-01 00:00:00'),
 Timestamp('2026-01-01 00:00:00'),
 Timestamp('2026-02-01 00:00:00'),
 Timestamp('2026-03-01 00:00:00')]

In [85]:
# validate if data is only processed for intended customer

unique_customers = pandas_df['CustomerID'].unique()
unique_PodIDs = pandas_df['PodID'].unique()

unique_customers, unique_PodIDs

(array(['7397925811', '7999245034'], dtype=object),
 array(['0736171562', '3289712697', '3750837137', '7027193072',
        '9849938574'], dtype=object))

In [88]:
Forecast_Method_Name

'SARIMA'

In [95]:
Hyper_Parameters

'(6,3,1)(6,3,2,3)'

In [93]:
order, seasonal_order = re.findall(r'\(.*?\)',Hyper_Parameters)   #['(1,1,1),(1,1,1,2)']
order, seasonal_order

('(6,3,1)', '(6,3,2,3)')

In [98]:
# Remove parentheses and split the parameters by commas to extract individual elements.
order = order.strip('()')
order_parameters = order.split(',')

seasonal_order=seasonal_order.strip('()')
seasonal_order_parameters = seasonal_order.split(',')



# Assign the values to the variables
p = int(order_parameters[0])
d = int(order_parameters[1])
q = int(order_parameters[2])


# Convert extracted string parameters into integers and assign them to corresponding variables. 
 
s_p = int(seasonal_order_parameters[0])
s_d = int(seasonal_order_parameters[1])
s_q = int(seasonal_order_parameters[2])
s_m = int(seasonal_order_parameters[3])

#print(f"(Order_Paramaters {p,d,q}  Seasonal_Paramaters {s_p,s_d,s_q,s_m}")  

In [100]:
order_parameters

['6', '3', '1']

In [105]:
plot_columns = selected_columns[2:]   
plot_columns

['PeakConsumption',
 'StandardConsumption',
 'OffPeakConsumption',
 'Block1Consumption',
 'Block2Consumption',
 'Block3Consumption',
 'Block4Consumption',
 'NonTOUConsumption']